# MIMIC-Multimodal: Master Dataset Generation

The data structure of this implementation is inspired by 
Soenksen, L. R. et al. Integrated multimodal artificial intelligence framework for healthcare applications. npj Digit. Med. 5, 149 (2022).

For more details, please visit:
https://physionet.org/content/haim-multimodal/1.0.1/

For data access and description, please visit:
https://mimic.mit.edu/

MIMIC-IV https://physionet.org/content/mimiciv/2.2/#files-panel \
MIMIC-CXR https://physionet.org/content/mimic-cxr/2.0.0/#files-panel \
MIMIC-CXR-JPG https://physionet.org/content/mimic-cxr-jpg/2.0.0/ \
MIMIC-IV-Note https://physionet.org/content/mimic-iv-note/2.2/note/#files-panel 


In [3]:
import numpy as np
import pandas as pd
import pickle
import datetime as dt
from pandasql import sqldf
from data_utils import *

import warnings
warnings.filterwarnings("ignore")

In [2]:
# File path

# MIMIC-IV
mimiciv_hosp_path = '../mimic-iv-2.2/hosp/'
mimiciv_icu_path = '../mimic-iv-2.2/icu/'
# MIMIV-CXR & MIMIC-CXR-JPG
mimiciv_cxr_path = '../mimic-cxr/'
mimiciv_cxr_jpg_path = '../mimic-cxr-jpg/'
# MIMIC-IV-Note
mimiciv_note_path = '../mimic-iv-note/note/'

## Load Data
For memory efficiency, we first load all files into  **Dask DataFrames** \
When processing is required, we call **ddf.compute()** to convert the data into a Pandas DataFrame

### read files by folder

In [3]:
# MIMIC-IV hosp module
dfs_hosp = {}
dfs_hosp = read_folder(dfs_hosp, mimiciv_hosp_path)

Read files in folder ../mimic-iv-2.2/hosp/


100%|██████████| 21/21 [11:24<00:00, 32.59s/it]


In [4]:
# MIMIC-IV icu module
dfs_icu = {}
# Read large dataframes into pandas dataframe since computing such dask dataframe requires a great amount of time and memory
dfs_icu['chartevents'] = pd.read_csv(mimiciv_icu_path+'chartevents.csv.gz', compression='gzip')
dfs_icu = read_folder(dfs_icu,mimiciv_icu_path)

Read files in folder ../mimic-iv-2.2/icu/


100%|██████████| 8/8 [01:11<00:00,  8.92s/it]


In [19]:
# MIMIC-IV-CXR
dfs_cxr = {}
dfs_cxr = read_folder(dfs_cxr, mimiciv_cxr_path)
dfs_cxr_jpg = {}
dfs_cxr_jpg = read_folder(dfs_cxr_jpg, mimiciv_cxr_jpg_path)

Read files in folder ../mimic-cxr/


100%|██████████| 6/6 [00:01<00:00,  5.65it/s]


Read files in folder ../mimic-cxr-jpg/


100%|██████████| 5/5 [00:01<00:00,  3.05it/s]


In [6]:
# MIMIC-IV-Notes
dfs_note = {}
dfs_note = read_folder(dfs_note, mimiciv_note_path)

Read files in folder ../mimic-iv-note/note/


100%|██████████| 4/4 [02:18<00:00, 34.53s/it]


### datetime conversion

In [7]:
# Hosp
dfs_hosp = convert_datetime(dfs_hosp)
# ICU
dfs_icu = convert_datetime(dfs_icu)
# Note
dfs_note = convert_datetime(dfs_note)

In [20]:
# convert time-related variables in CXR metadata
dfs_cxr_jpg['mimic-cxr-2.0.0-metadata'] = dfs_cxr_jpg['mimic-cxr-2.0.0-metadata'].compute()
df = dfs_cxr_jpg['mimic-cxr-2.0.0-metadata']
df['StudyDate'] = df['StudyDate'].astype('int')
df['StudyDate'] = pd.to_datetime(df['StudyDate'],format='%Y%m%d')
df['StudyTime'] = df.apply(lambda x : '%#010.3f' % x['StudyTime'] ,1)
df['StudyTime'] = pd.to_datetime(df['StudyTime'], format='%H%M%S.%f').dt.strftime('%H%M%S')
df['StudyTime'] = pd.to_datetime(df['StudyTime'], format='%H%M%S').dt.time
df['StudyDatetime'] = df.apply(lambda r : dt.datetime.combine(r['StudyDate'],r['StudyTime']),1)

## ID combinations

### get ID lists from each module

In [31]:
## MIMIC-IV
dfs_hosp['admissions'] = dfs_hosp['admissions'].compute()
dfs_icu['icustays'] = dfs_icu['icustays'].compute()
## MIMIC-IV CXR
dfs_cxr['cxr-record-list'] = dfs_cxr['cxr-record-list'].compute()
## MIMIC-IV Note
dfs_note['discharge'] = dfs_note['discharge'].compute()
dfs_note['radiology'] = dfs_note['radiology'].compute()

In [46]:
# Get all combinations of IDs in ICU module
icu_info = dfs_icu['icustays'][['subject_id','hadm_id','stay_id','intime','outtime']].copy()
icu_info = icu_info.merge(dfs_hosp['admissions'][['subject_id','hadm_id','admittime','dischtime','edregtime','edouttime']],
                          on=['subject_id','hadm_id'],how='left')
icu_info['earliest_intime'] = icu_info[['intime','admittime','edregtime']].min(axis=1) # earliest entering time for each hospitalization
# Get all combination of IDs in CXR module
cxr_info = dfs_cxr_jpg['mimic-cxr-2.0.0-metadata'][['subject_id','study_id','dicom_id','StudyDate','StudyTime','StudyDatetime']].copy()
# Get all combinations of IDs in Note module
note_ds_info = dfs_note['discharge'][['note_id','subject_id','hadm_id','charttime']].copy()
note_ds_info.rename(columns={'note_id':'ds_note_id','charttime':'ds_charttime'},inplace=True)
note_rad_info = dfs_note['radiology'][['note_id','subject_id','hadm_id','charttime']].copy()
note_rad_info.rename(columns={'note_id':'rad_note_id','charttime':'rad_charttime'},inplace=True)

### merge IDs by key identifiers and time

In [52]:
pysqldf = lambda q: sqldf(q, globals())

In [58]:
# For radiology reports and chest X-ray, we combine the data also by time

## Join on MIMIC-IV,MIMIC-CXR and MIMICIV-Note
sql_query = """
select distinct key_subject_id as subject_id,key_hadm_id as hadm_id,stay_id,study_id,dicom_id,ds_note_id,rad_note_id
from 
(
    select subject_id as key_subject_id,hadm_id as key_hadm_id,stay_id,intime,outtime,admittime,dischtime,earliest_intime
    from icu_info
) as i
left join cxr_info c
on i.key_subject_id = c.subject_id and c.StudyDatetime >= i.earliest_intime and c.StudyDatetime <= i.outtime
left join note_ds_info ds
on i.key_subject_id = ds.subject_id and i.key_hadm_id = ds.hadm_id
left join note_rad_info rad
on i.key_subject_id = rad.subject_id and i.key_hadm_id = rad.hadm_id and rad.rad_charttime >= i.earliest_intime and rad.rad_charttime <= i.outtime
"""
list_ids = pysqldf(sql_query)

In [84]:
list_ids

,subject_id,hadm_id,stay_id,study_id,dicom_id,ds_note_id,rad_note_id
0,10000032.0,29079034.0,39553978.0,53911762.0,68b5c4b1-227d0485-9cc38c3f-7b84ab51-4b472714,10000032-DS-23,10000032-RR-44
1,10000032.0,29079034.0,39553978.0,53911762.0,68b5c4b1-227d0485-9cc38c3f-7b84ab51-4b472714,10000032-DS-23,10000032-RR-45
2,10000032.0,29079034.0,39553978.0,53911762.0,fffabebf-74fd3a1f-673b6b41-96ec0ac9-2ab69818,10000032-DS-23,10000032-RR-44
3,10000032.0,29079034.0,39553978.0,53911762.0,fffabebf-74fd3a1f-673b6b41-96ec0ac9-2ab69818,10000032-DS-23,10000032-RR-45
4,10000980.0,26913865.0,39765666.0,51967283.0,943486a3-b3fa9ff7-50f5a769-7a62fcbb-f39b6da4,10000980-DS-21,10000980-RR-62
...,...,...,...,...,...,...,...
2142192,19999987.0,23865745.0,36195440.0,58971208.0,1a1fe7e3-cbac5d93-b339aeda-86bb86b5-4f31e82e,19999987-DS-2,19999987-RR-16
2142193,19999987.0,23865745.0,36195440.0,58971208.0,1a1fe7e3-cbac5d93-b339aeda-86bb86b5-4f31e82e,19999987-DS-2,19999987-RR-17
2142194,19999987.0,23865745.0,36195440.0,58971208.0,1a1fe7e3-cbac5d93-b339aeda-86bb86b5-4f31e82e,19999987-DS-2,19999987-RR-18
2142195,19999987.0,23865745.0,36195440.0,58971208.0,1a1fe7e3-cbac5d93-b339aeda-86bb86b5-4f31e82e,19999987-DS-2,19999987-RR-19


In [77]:
# key identifiers
key_ids = list_ids[['subject_id','hadm_id','stay_id']].drop_duplicates().reset_index(drop=True)
key_ids

,subject_id,hadm_id,stay_id
0,10000032.0,29079034.0,39553978.0
1,10000980.0,26913865.0,39765666.0
2,10001217.0,24597018.0,37067082.0
3,10001217.0,27703517.0,34592300.0
4,10001725.0,25563031.0,31205490.0
...,...,...,...
73176,19999442.0,26785317.0,32336619.0
73177,19999625.0,25304202.0,31070865.0
73178,19999828.0,25744818.0,36075953.0
73179,19999840.0,21033226.0,38978960.0


### summary

In [68]:
print('For patients admitted to ICU')
print('Number of unique patients:',list_ids['subject_id'].nunique())
print('Number of unique hospital admissions:',list_ids['hadm_id'].nunique())
print('Number of unique ICU stays:',list_ids['stay_id'].nunique())
print('Number of unique chest xray studies:',list_ids['study_id'].nunique())
print('Number of unique chest xray images:',list_ids['dicom_id'].nunique())
print('Number of unique discharge summaries:',list_ids['ds_note_id'].nunique())
print('Number of unique radiology reports:',list_ids['rad_note_id'].nunique())

For patients admitted to ICU
Number of unique patients: 50920
Number of unique hospital admissions: 66239
Number of unique ICU stays: 73181
Number of unique chest xray studies: 72180
Number of unique chest xray images: 86661
Number of unique discharge summaries: 65330
Number of unique radiology reports: 412664


## Extract information for each unique ICU stay

### functions

In [112]:
## MIMICIV PATIENT CLASS STRUCTURE
class Patient_ICU(object):
    def __init__(self, core, admissions, patients, transfers, diagnoses_icd, procedures_icd, drgcodes,
                 services, labevents, hcpcsevents, microbiologyevents, emar, poe, prescriptions, 
                 icustays, procedureevents, outputevents, inputevents, datetimeevents, chartevents,ingredientevents,
                 cxr_split, cxr_metadata, cxr_chexpert, cxr_negbio, cxr_image_path, cxr_text_path, dsnotes, radnotes):
        
        ## CORE
        self.core = core
        
        ## HOSP
        self.admissions = admissions
        self.patients = patients
        self.transfers = transfers
        self.diagnoses_icd = diagnoses_icd
        self.procedures_icd = procedures_icd
        self.drgcodes = drgcodes
        self.services = services
        self.labevents = labevents
        self.hcpcsevents = hcpcsevents
        self.microbiologyevents = microbiologyevents
        self.emar = emar
        self.poe = poe
        self.prescriptions = prescriptions
        
        ## ICU
        self.icustays = icustays
        self.procedureevents = procedureevents
        self.outputevents = outputevents
        self.inputevents = inputevents
        self.datetimeevents = datetimeevents
        self.chartevents = chartevents
        self.ingredientevents = ingredientevents
        
        ## CXR
        # JPG
        self.cxr_split = cxr_split
        self.cxr_metadata = cxr_metadata
        self.cxr_chexpert = cxr_chexpert
        self.cxr_negbio = cxr_negbio
        # DICOM
        self.cxr_image_path = cxr_image_path
        self.cxr_text_path = cxr_text_path
        
        ## NOTES
        self.dsnotes = dsnotes
        self.radnotes = radnotes

In [117]:
# Get full MIMIC-IV patient records using key_identifiers
def get_patient_icustay(key_subject_id, key_hadm_id, key_stay_id):
    """
    Inputs:
    key_subject_id -> subject_id is unique to a patient
    key_hadm_id    -> hadm_id is unique to a patient hospital stay
    key_stay_id    -> stay_id is unique to a patient ward stay
    Outputs:
    Patient_ICUstay -> ICU patient stay structure
    """
    # Data Extraction
    
    ## Table of identifiers
    df_core = list_ids[(list_ids.subject_id == key_subject_id) & (list_ids.hadm_id == key_hadm_id) & 
                       (list_ids.stay_id == key_stay_id)]
    
    ## Hosp - Tables are merged based on subject_id & hadm_id
    # Since miscellaneous information in OMR table is less detailed than in chartevents table, 
    # thus information from OMR table will not be included
    df_admissions = dfs_hosp['admissions'][(dfs_hosp['admissions'].subject_id == key_subject_id) & 
                                           (dfs_hosp['admissions'].hadm_id == key_hadm_id)]
    df_patients = dfs_hosp['patients'][(dfs_hosp['patients'].subject_id == key_subject_id)]
    df_transfers = dfs_hosp['transfers'][(dfs_hosp['transfers'].subject_id == key_subject_id) & 
                                         (dfs_hosp['transfers'].hadm_id == key_hadm_id)]
    df_diagnoses_icd = dfs_hosp['diagnoses_icd'][(dfs_hosp['diagnoses_icd'].subject_id == key_subject_id) &
                                                 (dfs_hosp['diagnoses_icd'].hadm_id == key_hadm_id)]
    df_diagnoses_icd = df_diagnoses_icd.merge(dfs_hosp['d_icd_diagnoses'],
                                              how='left', on=['icd_code', 'icd_version'])
    df_procedures_icd = dfs_hosp['procedures_icd'][(dfs_hosp['procedures_icd'].subject_id == key_subject_id) & 
                                                   (dfs_hosp['procedures_icd'].hadm_id == key_hadm_id)]
    df_procedures_icd = df_procedures_icd.merge(dfs_hosp['d_icd_procedures'], 
                                                how='left', on=['icd_code', 'icd_version'])
    df_drgcodes = dfs_hosp['drgcodes'][(dfs_hosp['drgcodes'].subject_id == key_subject_id) & 
                                       (dfs_hosp['drgcodes'].hadm_id == key_hadm_id)]
    df_services = dfs_hosp['services'][(dfs_hosp['services'].subject_id == key_subject_id) & 
                                       (dfs_hosp['services'].hadm_id == key_hadm_id)]
    df_labevents = dfs_hosp['labevents'][(dfs_hosp['labevents'].subject_id == key_subject_id) & 
                                         (dfs_hosp['labevents'].hadm_id == key_hadm_id)]
    df_labevents = df_labevents.merge(dfs_hosp['d_labitems'], how='left',on='itemid')
    df_hcpcsevents = dfs_hosp['hcpcsevents'][(dfs_hosp['hcpcsevents'].subject_id == key_subject_id) & 
                                             (dfs_hosp['hcpcsevents'].hadm_id == key_hadm_id)]
    df_hcpcsevents = df_hcpcsevents.merge(dfs_hosp['d_hcpcs'], how='left',
                                          left_on='hcpcs_cd',right_on='code')
    df_microbiologyevents = dfs_hosp['microbiologyevents'][(dfs_hosp['microbiologyevents'].subject_id == key_subject_id) & 
                                                           (dfs_hosp['microbiologyevents'].hadm_id == key_hadm_id)]
    df_emar = dfs_hosp['emar'][(dfs_hosp['emar'].subject_id == key_subject_id) & 
                               (dfs_hosp['emar'].hadm_id == key_hadm_id)]
    df_emar = df_emar.merge(dfs_hosp['emar_detail'], how='left', on='emar_id' )
    df_poe = dfs_hosp['poe'][(dfs_hosp['poe'].subject_id == key_subject_id) & (dfs_hosp['poe'].hadm_id == key_hadm_id)]
    df_poe = df_poe.merge(dfs_hosp['poe_detail'], how='left', on='poe_id')
    df_prescriptions = dfs_hosp['prescriptions'][(dfs_hosp['prescriptions'].subject_id == key_subject_id) & 
                                                 (dfs_hosp['prescriptions'].hadm_id == key_hadm_id)]
    df_prescriptions = df_prescriptions.merge(dfs_hosp['pharmacy'], how='left', on='pharmacy_id')
    
    ## ICU - Tables are merged based on subject_id & hadm_id & stay_id
    df_icustays = dfs_icu['icustays'][(dfs_icu['icustays'].subject_id == key_subject_id) & 
                                      (dfs_icu['icustays'].hadm_id == key_hadm_id) & 
                                      (dfs_icu['icustays'].stay_id == key_stay_id)]
    df_procedureevents = dfs_icu['procedureevents'][(dfs_icu['procedureevents'].subject_id == key_subject_id) & 
                                                    (dfs_icu['procedureevents'].hadm_id == key_hadm_id) & 
                                                    (dfs_icu['procedureevents'].stay_id == key_stay_id)]
    df_outputevents = dfs_icu['outputevents'][(dfs_icu['outputevents'].subject_id == key_subject_id) & 
                                              (dfs_icu['outputevents'].hadm_id == key_hadm_id) & 
                                              (dfs_icu['outputevents'].stay_id == key_stay_id)]
    df_inputevents = dfs_icu['inputevents'][(dfs_icu['inputevents'].subject_id == key_subject_id) & 
                                            (dfs_icu['inputevents'].hadm_id == key_hadm_id) & 
                                            (dfs_icu['inputevents'].stay_id == key_stay_id)]
    df_datetimeevents = dfs_icu['datetimeevents'][(dfs_icu['datetimeevents'].subject_id == key_subject_id) & 
                                                  (dfs_icu['datetimeevents'].hadm_id == key_hadm_id) & 
                                                  (dfs_icu['datetimeevents'].stay_id == key_stay_id)]
    df_chartevents = dfs_icu['chartevents'][(dfs_icu['chartevents'].subject_id == key_subject_id) & 
                                            (dfs_icu['chartevents'].hadm_id == key_hadm_id) & 
                                            (dfs_icu['chartevents'].stay_id == key_stay_id)]
    df_ingredientevents = dfs_icu['ingredientevents'][(dfs_icu['ingredientevents'].subject_id == key_subject_id) & 
                                                      (dfs_icu['ingredientevents'].hadm_id == key_hadm_id) & 
                                                      (dfs_icu['ingredientevents'].stay_id == key_stay_id)]
    # Merge descriptions into each table
    df_procedureevents = df_procedureevents.merge(dfs_icu['d_items'], how='left', on='itemid')
    df_outputevents = df_outputevents.merge(dfs_icu['d_items'], how='left', on='itemid')
    df_inputevents = df_inputevents.merge(dfs_icu['d_items'], how='left', on='itemid')
    df_datetimeevents = df_datetimeevents.merge(dfs_icu['d_items'], how='left', on='itemid')
    df_chartevents = df_chartevents.merge(dfs_icu['d_items'], how='left', on='itemid')
    df_ingredientevents = df_ingredientevents.merge(dfs_icu['d_items'], how='left', on='itemid')
    
    ## CXR
    # Get lists of study_id and dicom_id for each ICU stay
    study_id_list = df_core['study_id'].unique()
    dicom_id_list = df_core['dicom_id'].unique()
    # Extract tables from MIMIC-CXR
    df_cxr_image_path = dfs_cxr['cxr-record-list'][(dfs_cxr['cxr-record-list'].subject_id == key_subject_id) &
                                                   (dfs_cxr['cxr-record-list'].study_id.isin(study_id_list)) &
                                                   (dfs_cxr['cxr-record-list'].dicom_id.isin(dicom_id_list))]
    df_cxr_text_path = dfs_cxr['cxr-study-list'][(dfs_cxr['cxr-study-list'].subject_id == key_subject_id) &
                                                   (dfs_cxr['cxr-study-list'].study_id.isin(study_id_list))]
    # Extract tables from MIMIC-CXR-JPG
    df_cxr_metadata = dfs_cxr_jpg['mimic-cxr-2.0.0-metadata'][(dfs_cxr_jpg['mimic-cxr-2.0.0-metadata'].subject_id == key_subject_id) &
                                                              (dfs_cxr_jpg['mimic-cxr-2.0.0-metadata'].study_id.isin(study_id_list)) &
                                                              (dfs_cxr_jpg['mimic-cxr-2.0.0-metadata'].dicom_id.isin(dicom_id_list))]
    df_cxr_chexpert = dfs_cxr_jpg['mimic-cxr-2.0.0-chexpert'][(dfs_cxr_jpg['mimic-cxr-2.0.0-chexpert'].subject_id == key_subject_id) &
                                                              (dfs_cxr_jpg['mimic-cxr-2.0.0-chexpert'].study_id.isin(study_id_list))]
    df_cxr_negbio = dfs_cxr_jpg['mimic-cxr-2.0.0-negbio'][(dfs_cxr_jpg['mimic-cxr-2.0.0-negbio'].subject_id == key_subject_id) & 
                                                          (dfs_cxr_jpg['mimic-cxr-2.0.0-negbio'].study_id.isin(study_id_list))]
    df_cxr_split = dfs_cxr_jpg['mimic-cxr-2.0.0-split'][(dfs_cxr_jpg['mimic-cxr-2.0.0-split'].subject_id == key_subject_id) &
                                                        (dfs_cxr_jpg['mimic-cxr-2.0.0-split'].study_id.isin(study_id_list)) &
                                                        (dfs_cxr_jpg['mimic-cxr-2.0.0-split'].dicom_id.isin(dicom_id_list))]
    
    ## Notes
    ds_note_id_list = df_core['ds_note_id'].unique()
    rad_note_id_list = df_core['rad_note_id'].unique()
    df_dsnotes = dfs_note['discharge'][(dfs_note['discharge'].subject_id == key_subject_id) &
                                       (dfs_note['discharge'].hadm_id == key_hadm_id) &
                                       (dfs_note['discharge'].note_id.isin(ds_note_id_list))]
    df_radnotes = dfs_note['radiology'][(dfs_note['radiology'].subject_id == key_subject_id) &
                                        (dfs_note['radiology'].hadm_id == key_hadm_id) &
                                        (dfs_note['radiology'].note_id.isin(rad_note_id_list))]
    df_radnotes = df_radnotes.merge(dfs_note['radiology_detail'], how='left', on='note_id')
        
    # Create patient object and return
    Patient_ICUstay = Patient_ICU(df_core, df_admissions, df_patients, df_transfers, df_diagnoses_icd, df_procedures_icd, df_drgcodes,
                                  df_services, df_labevents, df_hcpcsevents, df_microbiologyevents, df_emar, df_poe, df_prescriptions, 
                                  df_icustays, df_procedureevents, df_outputevents, df_inputevents, df_datetimeevents, df_chartevents, df_ingredientevents,
                                  df_cxr_split, df_cxr_metadata, df_cxr_chexpert, df_cxr_negbio, df_cxr_image_path, df_cxr_text_path, 
                                  df_dsnotes, df_radnotes)
     
    return Patient_ICUstay

In [152]:
# Extract all single ICU stay records
def generate_master_dataset(key_ids, storage_path):
    # Inputs:
    #   key_ids -> Dataframe with all unique available records by key identifiers
    #   storage_path -> Path to structured MIMIC IV databases in pickle files
    
    # Outputs:
    #   nfiles -> Number of single patient files produced
    
    # Extract information for patient
    nfiles = len(key_ids)
    with tqdm(total = nfiles) as pbar:

        #Iterate through all patients
        for _, content in key_ids.iterrows():
            key_subject_id = content['subject_id']
            key_hadm_id = content['hadm_id']
            key_stay_id = content['stay_id']
            
            # Save objects
            filename = f'ICUstay_{int(key_stay_id)}'+'.pkl'
            icustay = get_patient_icustay(key_subject_id,key_hadm_id,key_stay_id)
            pickle.dump(icustay,open(storage_path+filename,'wb'))
            
            # Update process bar
            pbar.update(1)

### extract and save patient ICU stay information

In [125]:
# dfs_icu['d_items'] = dfs_icu['d_items'].compute()
# dfs_note['radiology_detail'] = dfs_note['radiology_detail'].compute()

In [157]:
ICU_path = 'D:/Master Dataset/'
generate_master_dataset(key_ids=key_ids[60000:],storage_path=ICU_path)

100%|██████████| 13181/13181 [11:08:32<00:00,  3.04s/it]  


~3s/item

## Examples

In [4]:
ICUstay_test = pickle.load(open('D:/Master Dataset/ICUstay_30000646.pkl','rb'))

In [9]:
ICUstay_test.chartevents[ICUstay_test.chartevents['label']=='Respiratory Rate']

,subject_id,hadm_id,stay_id,caregiver_id,charttime,storetime,itemid,value,valuenum,valueuom,warning,label,abbreviation,linksto,category,unitname,param_type,lownormalvalue,highnormalvalue
4,12207593,22795209,30000646,2234.0,2194-05-01 06:00:00,2194-05-01 08:43:00,220210,23,23.0,insp/min,0.0,Respiratory Rate,RR,chartevents,Respiratory,insp/min,Numeric,NaN,NaN
10,12207593,22795209,30000646,2234.0,2194-05-01 07:00:00,2194-05-01 08:43:00,220210,20,20.0,insp/min,0.0,Respiratory Rate,RR,chartevents,Respiratory,insp/min,Numeric,NaN,NaN
17,12207593,22795209,30000646,2234.0,2194-05-01 08:00:00,2194-05-01 08:43:00,220210,25,25.0,insp/min,0.0,Respiratory Rate,RR,chartevents,Respiratory,insp/min,Numeric,NaN,NaN
110,12207593,22795209,30000646,2234.0,2194-05-01 09:00:00,2194-05-01 09:28:00,220210,30,30.0,insp/min,0.0,Respiratory Rate,RR,chartevents,Respiratory,insp/min,Numeric,NaN,NaN
116,12207593,22795209,30000646,2234.0,2194-05-01 10:00:00,2194-05-01 10:58:00,220210,24,24.0,insp/min,0.0,Respiratory Rate,RR,chartevents,Respiratory,insp/min,Numeric,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4093,12207593,22795209,30000646,98277.0,2194-05-01 01:00:00,2194-05-01 05:53:00,220210,18,18.0,insp/min,0.0,Respiratory Rate,RR,chartevents,Respiratory,insp/min,Numeric,NaN,NaN
4100,12207593,22795209,30000646,98277.0,2194-05-01 02:00:00,2194-05-01 05:53:00,220210,16,16.0,insp/min,0.0,Respiratory Rate,RR,chartevents,Respiratory,insp/min,Numeric,NaN,NaN
4107,12207593,22795209,30000646,98277.0,2194-05-01 03:00:00,2194-05-01 05:53:00,220210,18,18.0,insp/min,0.0,Respiratory Rate,RR,chartevents,Respiratory,insp/min,Numeric,NaN,NaN
4114,12207593,22795209,30000646,98277.0,2194-05-01 04:00:00,2194-05-01 05:53:00,220210,16,16.0,insp/min,0.0,Respiratory Rate,RR,chartevents,Respiratory,insp/min,Numeric,NaN,NaN


In [203]:
ICUstay_test.cxr_image_path

,subject_id,study_id,dicom_id,path
82844,12207593.0,56369155.0,7f124ef7-a1714162-b7e9da18-5862d297-3fd8f3a9,files/p12/p12207593/s56369155/7f124ef7-a1714162-b7e9da18-5862d297-3fd8f3a9.dcm
82845,12207593.0,57440284.0,49d77ea6-5f9a013f-d0e6c33e-bb3aae40-88d57b43,files/p12/p12207593/s57440284/49d77ea6-5f9a013f-d0e6c33e-bb3aae40-88d57b43.dcm
82846,12207593.0,57520242.0,40f24d85-94293a52-b7e4961d-d2ad9561-864efe77,files/p12/p12207593/s57520242/40f24d85-94293a52-b7e4961d-d2ad9561-864efe77.dcm
82847,12207593.0,57688955.0,f725e388-3b557432-84d5882b-4c406d73-68132627,files/p12/p12207593/s57688955/f725e388-3b557432-84d5882b-4c406d73-68132627.dcm


In [183]:
display(ICUstay_test.cxr_metadata.head())
display(ICUstay_test.cxr_image_path.head())
display(ICUstay_test.cxr_text_path.head())

,dicom_id,subject_id,study_id,PerformedProcedureStepDescription,ViewPosition,Rows,Columns,StudyDate,StudyTime,ProcedureCodeSequence_CodeMeaning,ViewCodeSequence_CodeMeaning,PatientOrientationCodeSequence_CodeMeaning,StudyDatetime
82844,7f124ef7-a1714162-b7e9da18-5862d297-3fd8f3a9,12207593.0,56369155.0,CHEST (PORTABLE AP),AP,2544.0,3056.0,2194-04-30,08:33:53,CHEST (PORTABLE AP),antero-posterior,Erect,2194-04-30 08:33:53
82845,49d77ea6-5f9a013f-d0e6c33e-bb3aae40-88d57b43,12207593.0,57440284.0,CHEST (PORTABLE AP),AP,2544.0,3056.0,2194-04-29,06:25:10,CHEST (PORTABLE AP),antero-posterior,Erect,2194-04-29 06:25:10
82846,40f24d85-94293a52-b7e4961d-d2ad9561-864efe77,12207593.0,57520242.0,CHEST (PORTABLE AP),AP,2539.0,3050.0,2194-05-02,10:45:39,CHEST (PORTABLE AP),antero-posterior,Erect,2194-05-02 10:45:39
82847,f725e388-3b557432-84d5882b-4c406d73-68132627,12207593.0,57688955.0,CHEST (PORTABLE AP),AP,2539.0,3050.0,2194-05-01,08:06:11,CHEST (PORTABLE AP),antero-posterior,Erect,2194-05-01 08:06:11


,subject_id,study_id,dicom_id,path
82844,12207593.0,56369155.0,7f124ef7-a1714162-b7e9da18-5862d297-3fd8f3a9,files/p12/p12207593/s56369155/7f124ef7-a1714162-b7e9da18-5862d297-3fd8f3a9.dcm
82845,12207593.0,57440284.0,49d77ea6-5f9a013f-d0e6c33e-bb3aae40-88d57b43,files/p12/p12207593/s57440284/49d77ea6-5f9a013f-d0e6c33e-bb3aae40-88d57b43.dcm
82846,12207593.0,57520242.0,40f24d85-94293a52-b7e4961d-d2ad9561-864efe77,files/p12/p12207593/s57520242/40f24d85-94293a52-b7e4961d-d2ad9561-864efe77.dcm
82847,12207593.0,57688955.0,f725e388-3b557432-84d5882b-4c406d73-68132627,files/p12/p12207593/s57688955/f725e388-3b557432-84d5882b-4c406d73-68132627.dcm


,subject_id,study_id,path
50237,12207593.0,56369155.0,files/p12/p12207593/s56369155.txt
50238,12207593.0,57440284.0,files/p12/p12207593/s57440284.txt
50239,12207593.0,57520242.0,files/p12/p12207593/s57520242.txt
50240,12207593.0,57688955.0,files/p12/p12207593/s57688955.txt


In [141]:
for attribute, value in ICUstay_test.__dict__.items():
    print(attribute)
    display(value)

core


,subject_id,hadm_id,stay_id,study_id,dicom_id,ds_note_id,rad_note_id
130,10002155.0,28994087.0,31090461.0,NaN,None,10002155-DS-10,10002155-RR-50
131,10002155.0,28994087.0,31090461.0,NaN,None,10002155-DS-10,10002155-RR-52
132,10002155.0,28994087.0,31090461.0,NaN,None,10002155-DS-10,10002155-RR-53
133,10002155.0,28994087.0,31090461.0,NaN,None,10002155-DS-10,10002155-RR-54
134,10002155.0,28994087.0,31090461.0,NaN,None,10002155-DS-10,10002155-RR-55
135,10002155.0,28994087.0,31090461.0,NaN,None,10002155-DS-10,10002155-RR-56


admissions


,subject_id,hadm_id,admittime,dischtime,deathtime,admission_type,admit_provider_id,admission_location,discharge_location,insurance,language,marital_status,race,edregtime,edouttime,hospital_expire_flag
106,10002155.0,28994087.0,2130-09-23 21:59:00,2130-09-29 18:55:00,NaT,EW EMER.,P3529J,EMERGENCY ROOM,HOME HEALTH CARE,Other,ENGLISH,MARRIED,WHITE,2130-09-23 19:59:00,2130-09-24 00:50:00,0.0


patients


...

,subject_id,study_id,path
npartitions=1,,,
,float64,float64,string
,...,...,...


dsnotes


,note_id,subject_id,hadm_id,note_type,note_seq,charttime,storetime,text
74,10002155-DS-10,10002155.0,28994087.0,DS,10.0,2130-09-29,2130-10-01 16:46:00,\nName: ___ Unit No: ___\n...


radnotes


,note_id,subject_id_x,hadm_id,note_type,note_seq,charttime,storetime,text,subject_id_y,field_name,field_value,field_ordinal
0,10002155-RR-50,10002155.0,28994087.0,RR,50.0,2130-09-23 20:24:00,2130-09-23 21:49:00,INDICATION: History of lung cancer with decre...,10002155.0,exam_code,C12,1.0
1,10002155-RR-50,10002155.0,28994087.0,RR,50.0,2130-09-23 20:24:00,2130-09-23 21:49:00,INDICATION: History of lung cancer with decre...,10002155.0,exam_name,CHEST (PORTABLE AP),1.0
2,10002155-RR-52,10002155.0,28994087.0,RR,52.0,2130-09-24 00:03:00,2130-09-24 01:47:00,INDICATION: In need of heparin. Rule out met...,10002155.0,exam_code,Q111,1.0
3,10002155-RR-52,10002155.0,28994087.0,RR,52.0,2130-09-24 00:03:00,2130-09-24 01:47:00,INDICATION: In need of heparin. Rule out met...,10002155.0,exam_name,CT HEAD W/O CONTRAST,1.0
4,10002155-RR-53,10002155.0,28994087.0,RR,53.0,2130-09-25 04:18:00,2130-09-25 18:26:00,AP CHEST 4:42 A.M. ___\n\nHISTORY: Hypoxia an...,10002155.0,exam_code,C12,1.0
5,10002155-RR-53,10002155.0,28994087.0,RR,53.0,2130-09-25 04:18:00,2130-09-25 18:26:00,AP CHEST 4:42 A.M. ___\n\nHISTORY: Hypoxia an...,10002155.0,exam_name,CHEST (PORTABLE AP),1.0
6,10002155-RR-54,10002155.0,28994087.0,RR,54.0,2130-09-24 15:09:00,2130-09-24 19:56:00,HISTORY: ___ female with hypoxia.\n\nSTUDY: ...,10002155.0,exam_code,U614,1.0
7,10002155-RR-54,10002155.0,28994087.0,RR,54.0,2130-09-24 15:09:00,2130-09-24 19:56:00,HISTORY: ___ female with hypoxia.\n\nSTUDY: ...,10002155.0,exam_name,BILAT LOWER EXT VEINS,1.0
8,10002155-RR-55,10002155.0,28994087.0,RR,55.0,2130-09-26 03:52:00,2130-09-26 10:08:00,AP CHEST 4:16 A.M. ___ \n\nHISTORY: Lung ca...,10002155.0,exam_code,C12,1.0
9,10002155-RR-55,10002155.0,28994087.0,RR,55.0,2130-09-26 03:52:00,2130-09-26 10:08:00,AP CHEST 4:16 A.M. ___ \n\nHISTORY: Lung ca...,10002155.0,exam_name,CHEST (PORTABLE AP),1.0


In [140]:
with tqdm(total=len(ICUstay_test.__dict__.keys())) as pbar:
    for attribute, value in ICUstay_test.__dict__.items():
        if isinstance(value,dd.DataFrame):
            try:
                value.head(5)
            except Exception as e:
                print(attribute)
                print(e)  
            pbar.update(1)
        else:
            pbar.update(1)        

100%|██████████| 29/29 [23:11<00:00, 48.00s/it] 
